# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np
tstart = '2017-10-05T10:00:00'
tend = '2017-10-06T10:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/lunar_278to279.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-10-05 01:27:57,2017-10-05 02:23:46
1,2017-10-05 03:04:37,2017-10-05 04:00:25
2,2017-10-05 04:41:16,2017-10-05 05:37:05
3,2017-10-05 06:17:55,2017-10-05 07:13:44
4,2017-10-05 07:54:35,2017-10-05 08:50:23


# Use SkyField to get the location of Jupiter for each orbit:

This puts the output into the provided text file. This shows the *aim* time that was used to determine the pointing. You should slew while the source is occulted.

In [ ]:
from nustar_planning import lunar_planning
outfile = 'lunar_{0}_{1}_pointing.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )
lunar_planning.position(orbits, load_path ='../../data',
                 parallax_correction=True,
                 outfile=outfile, show=True)
print("Output is stored in: {}".format(outfile))

In [4]:
from nustar_planning.lunar_planning import position_shift



In [5]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)

position_shift(orbits, parallax_correction=True)

First position:
2017-10-05T01:27:57 RA: 4.63426  Dec: -2.31234
Dwell Time (sec): 770.00 Shift (arcmin): 6.02
2017-10-05T01:40:47 RA: 4.54100  Dec: -2.34965

Dwell Time (sec): 160.00 Shift (arcmin): 6.16
2017-10-05T01:43:27 RA: 4.43829  Dec: -2.34721

Dwell Time (sec): 135.00 Shift (arcmin): 6.24
2017-10-05T01:45:42 RA: 4.33429  Dec: -2.34184

Dwell Time (sec): 120.00 Shift (arcmin): 6.22
2017-10-05T01:47:42 RA: 4.23080  Dec: -2.33448

Dwell Time (sec): 110.00 Shift (arcmin): 6.15
2017-10-05T01:49:32 RA: 4.12870  Dec: -2.32559

Dwell Time (sec): 105.00 Shift (arcmin): 6.16
2017-10-05T01:51:17 RA: 4.02643  Dec: -2.31525

Dwell Time (sec): 100.00 Shift (arcmin): 6.05
2017-10-05T01:52:57 RA: 3.92614  Dec: -2.30376

Dwell Time (sec): 100.00 Shift (arcmin): 6.14
2017-10-05T01:54:37 RA: 3.82450  Dec: -2.29075

Dwell Time (sec): 100.00 Shift (arcmin): 6.15
2017-10-05T01:56:17 RA: 3.72294  Dec: -2.27633

Dwell Time (sec): 100.00 Shift (arcmin): 6.07
2017-10-05T01:57:57 RA: 3.62290  Dec: -2.2606

In [6]:
# PA == 30, Rmoon = 930 arcseconds, 6 arcminutes maximum drift (default) and
# a minimum dwell period of 200 seconds (default):
position_shift(orbits, parallax_correction=True, Rmoon=930*u.arcsec, 
              min_dwell=200*u.s, pa = 30*u.deg)


Dwell Time (sec): 770.00 Shift (arcmin): 6.02
2017-10-05T01:40:47 RA: 4.41174  Dec: -2.31226

Dwell Time (sec): 205.00 Shift (arcmin): 8.14
2017-10-05T01:44:12 RA: 4.27600  Dec: -2.30837

Dwell Time (sec): 205.00 Shift (arcmin): 10.21
2017-10-05T01:47:37 RA: 4.10604  Dec: -2.29744

Dwell Time (sec): 205.00 Shift (arcmin): 11.68
2017-10-05T01:51:02 RA: 3.91201  Dec: -2.27945

Dwell Time (sec): 205.00 Shift (arcmin): 12.48
2017-10-05T01:54:27 RA: 3.70544  Dec: -2.25473

Dwell Time (sec): 205.00 Shift (arcmin): 12.54
2017-10-05T01:57:52 RA: 3.49860  Dec: -2.22403

Dwell Time (sec): 205.00 Shift (arcmin): 11.88
2017-10-05T02:01:17 RA: 3.30376  Dec: -2.18841

Dwell Time (sec): 205.00 Shift (arcmin): 10.54
2017-10-05T02:04:42 RA: 3.13249  Dec: -2.14919

Dwell Time (sec): 205.00 Shift (arcmin): 8.61
2017-10-05T02:08:07 RA: 2.99492  Dec: -2.10788

Dwell Time (sec): 205.00 Shift (arcmin): 6.26
2017-10-05T02:11:32 RA: 2.89925  Dec: -2.06608

Dwell Time (sec): 520.00 Shift (arcmin): 6.01
2017-10-

In [ ]:
# Test PA = 0 vs PA 180
ra = 4.5574242917772985
dec = -2.349650003454101
test = SkyCoord(ra, dec, unit=u.deg)

ra = 4.5574242917772985
dec =-1.8274277812318789
test2 = SkyCoord(ra, dec, unit=u.deg)

dr = test.separation(test2)
dr

In [ ]:
# Test PA 90 vs PA 270
ra = 4.296139610238649
dec = -2.08853889234299
test = SkyCoord(ra, dec, unit=u.deg)
ra = 4.818708973315948 
dec =-2.08853889234299
test2 = SkyCoord(ra, dec, unit=u.deg)
test2.separation(test)